# GrimAge2LogA1C

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [17]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [18]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.GrimAge2LogA1C)

class GrimAge2LogA1C(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [19]:
model = pya.models.GrimAge2LogA1C()

## Define clock metadata

In [20]:
model.metadata["clock_name"] = 'grimage2loga1c'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, Ake T., et al. \"DNA methylation GrimAge version 2.\" Aging (Albany NY) 14.23 (2022): 9484."
model.metadata["doi"] = "https://doi.org/10.18632/aging.204434"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [21]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2_subcomponents.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./grimage2_subcomponents.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./grimage2.csv
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./datMiniAnnotation3_Gold.csv
|-----------> in progress: 100.0000%


## Load features

#### From CSV

In [22]:
df = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
df['Y.pred'].unique()

array(['DNAmGDF_15', 'DNAmB2M', 'DNAmCystatin_C', 'DNAmTIMP_1', 'DNAmadm',
       'DNAmpai_1', 'DNAmleptin', 'DNAmPACKYRS', 'DNAmlog.CRP',
       'DNAmlog.A1C'], dtype=object)

In [23]:
df = df[df['Y.pred'] == 'DNAmlog.A1C']
df['feature'] = df['var']
df['coefficient'] = df['beta']
model.features = ['age'] + df['feature'][2:].tolist()

In [24]:
df.head()

,Y.pred,var,beta,feature,coefficient
1929,DNAmlog.A1C,Intercept,1.613356,Intercept,1.613356
1930,DNAmlog.A1C,Age,0.000048,Age,0.000048
1931,DNAmlog.A1C,cg14260002,0.205684,cg14260002,0.205684
1932,DNAmlog.A1C,cg07592681,-0.132551,cg07592681,-0.132551
1933,DNAmlog.A1C,cg06959205,-0.008526,cg06959205,-0.008526


## Load weights into base model

#### Linear model

In [25]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'].iloc[0]])

#### Linear model

In [26]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [27]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = [65] + reference_df.loc[model.features[1:]]['gold'].tolist()

## Load preprocess and postprocess objects

In [28]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [29]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [30]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, Ake T., et al. "DNA methylation GrimAge version 2." Aging '
             '(Albany NY) 14.23 (2022): 9484.',
 'clock_name': 'grimage2loga1c',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.204434',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [65, 0.931162975482825, 0.902618842256319, 0.526434127110235, 0.809605421222521, 0.836130800392971, 0.0600816991722836, 0.332104407846093, 0.0512855773762465, 0.805395132732413, 0.565221968837601, 0.0752133473674328, 0.0979220649321982, 0.702649834869793, 0.0701168220050587, 0.733082005089516, 0.182279578592817, 0.768742491924122, 0.0587619174529237, 0.36823650486503, 0.812900664876887, 0.0463349565573039, 0.769921114503612, 0.442744708206467, 0.323099124395025, 0.067219447673

## Basic test

In [31]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[1.1562],
        [1.7227],
        [2.0204],
        [1.3346],
        [1.3498],
        [1.7763],
        [2.2459],
        [2.1019],
        [1.3236],
        [1.4237]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [32]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [33]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: grimage2_subcomponents.csv
Deleted file: datMiniAnnotation3_Gold.csv
Deleted file: grimage2.csv
